# Prediction of the wine dataset
The data conssts of 9 columns where variety is the output column where prediction is to be done. The propsed model is a cascading model which uses country,province,points,winery,price for prediction of variety.The dataset is loaded into the the variable data.

In [1]:
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import subprocess
subprocess.check_call(["python", '-m', 'pip', 'install', 'gensim'])
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from collections import Counter
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('data.csv')
print(data.head(2))
print(data.describe())
print(data.info())

       user_name    country  \
0            NaN  Australia   
1  @wawinereport         US   

                                        review_title  \
0  Andrew Peace 2007 Peace Family Vineyard Chardo...   
1  North by Northwest 2014 Red (Columbia Valley (...   

                                  review_description  points  price  \
0  Classic Chardonnay aromas of apple, pear and h...      83   10.0   
1  This wine is near equal parts Syrah and Merlot...      89   15.0   

          province              winery     variety  
0  Australia Other        Andrew Peace  Chardonnay  
1       Washington  North by Northwest   Red Blend  
             points         price
count  82657.000000  77088.000000
mean      88.544564     36.922232
std        3.099649     43.698346
min       80.000000      4.000000
25%       86.000000     18.000000
50%       88.000000     27.000000
75%       91.000000     45.000000
max      100.000000   3300.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 en

## Removing user name column and filling price column NaN value with mean

In [3]:
data['price'].fillna(value=data['price'].mean(), inplace=True)
data=data.drop(['user_name'], axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 entries, 0 to 82656
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             82622 non-null  object 
 1   review_title        82657 non-null  object 
 2   review_description  82657 non-null  object 
 3   points              82657 non-null  int64  
 4   price               82657 non-null  float64
 5   province            82622 non-null  object 
 6   winery              82657 non-null  object 
 7   variety             82657 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 5.0+ MB
None


In [4]:
print(len(data[data.isnull().any(axis=1)]))
data=data.dropna()
print(data.info())

35
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82622 entries, 0 to 82656
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             82622 non-null  object 
 1   review_title        82622 non-null  object 
 2   review_description  82622 non-null  object 
 3   points              82622 non-null  int64  
 4   price               82622 non-null  float64
 5   province            82622 non-null  object 
 6   winery              82622 non-null  object 
 7   variety             82622 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 5.7+ MB
None


In [ ]:
!pip install ploty
import plotly.express as px
wd_price = data.groupby('points')['price'].mean().reset_index()
wd_price.head()
fig = px.scatter(x=wd_price['points'], y=wd_price['price'], title = 'Price Distribution against Points')
fig.show()

## Performiing oridinal encoding withthe target variable and creating train test split

In [15]:
from sklearn.preprocessing import OrdinalEncoder
y = data['variety'].values
oe = OrdinalEncoder()
y = oe.fit_transform(y.reshape(-1,1))
X = data.drop(['variety'], axis=1)
print(X.head(3))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

     country                                       review_title  \
0  Australia  Andrew Peace 2007 Peace Family Vineyard Chardo...   
1         US  North by Northwest 2014 Red (Columbia Valley (...   
2      Italy                  Renato Ratti 2007 Conca  (Barolo)   

                                  review_description  points  price  \
0  Classic Chardonnay aromas of apple, pear and h...      83   10.0   
1  This wine is near equal parts Syrah and Merlot...      89   15.0   
2  Barolo Conca opens with inky dark concentratio...      94   80.0   

          province              winery  
0  Australia Other        Andrew Peace  
1       Washington  North by Northwest  
2         Piedmont        Renato Ratti  


## Using Tfidfvectorizer in review title and description 


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)

vectorizer.fit(X_train['review_title'].values)
X_train_title= vectorizer.transform(X_train['review_title'].values)
X_test_title= vectorizer.transform(X_test['review_title'].values)

print(X_train_title.shape)

print(X_test_title.shape)

(55356, 3581)
(27266, 3581)


In [17]:
vectorizer.fit(X_train['review_description'].values)
X_train_review= vectorizer.transform(X_train['review_description'].values)
X_test_review= vectorizer.transform(X_test['review_description'].values)

print(X_train_review.shape)

print(X_test_review.shape)

(55356, 6127)
(27266, 6127)


## Using CountVectorizer on categorical features : country, winery and province

In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['country'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_country = vectorizer.transform(X_train['country'].values)
X_test_country= vectorizer.transform(X_test['country'].values)

print("After vectorizations")
print(X_train_country.shape, y_train.shape)
print(X_test_country.shape, y_test.shape)
print(vectorizer.get_feature_names())

After vectorizations
(55356, 40) (55356, 1)
(27266, 40) (27266, 1)
['africa', 'argentina', 'australia', 'austria', 'brazil', 'bulgaria', 'canada', 'chile', 'croatia', 'cyprus', 'czech', 'england', 'france', 'georgia', 'germany', 'greece', 'hungary', 'israel', 'italy', 'lebanon', 'luxembourg', 'macedonia', 'mexico', 'moldova', 'morocco', 'new', 'peru', 'portugal', 'republic', 'romania', 'serbia', 'slovenia', 'south', 'spain', 'switzerland', 'turkey', 'ukraine', 'uruguay', 'us', 'zealand']


In [19]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['winery'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_winery = vectorizer.transform(X_train['winery'].values)
X_test_winery= vectorizer.transform(X_test['winery'].values)

print("After vectorizations")
print(X_train_winery.shape, y_train.shape)
print(X_test_winery.shape, y_test.shape)
print(vectorizer.get_feature_names())

After vectorizations
(55356, 11904) (55356, 1)
(27266, 11904) (27266, 1)
['100', '1000', '103', '10span', '1150', '12', '12c', '14', '1492', '15', '154', '1637', '16x20', '1789', '18401', '1848', '1850', '1852', '1856', '1868', '1877', '1882', '1898', '19', '1912', '1929', '21', '22', '24', '240', '25', '28', '29', '2hawk', '2nd', '2plank', '31st', '32', '33', '337', '34', '35', '351', '36', '38', '39', '3fools', '401k', '41', '413', '42', '44', '460', '46n118', '50', '511', '52', '585', '5os', '60', '66', '6cepas6', '7200', '75', '772', '786', '800', '868', '91', 'aaron', 'abacela', 'abad', 'abadia', 'abarbanel', 'abbadia', 'abbaye', 'abbazia', 'abbey', 'abbona', 'abbot', 'abeja', 'abele', 'abelis', 'abella', 'abelyce', 'aberrant', 'abiouness', 'abiqua', 'abrantes', 'abrigo', 'acacia', 'academy', 'acaibo', 'acate', 'accordini', 'aces', 'achaia', 'achaval', 'aciprestes', 'acker', 'ackerman', 'aconquija', 'acordeón', 'acorn', 'acquiesce', 'acre', 'acres', 'acrobat', 'acronym', 'act', 'a

In [20]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['province'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_province = vectorizer.transform(X_train['province'].values)
X_test_province= vectorizer.transform(X_test['province'].values)

print("After vectorizations")
print(X_train_province.shape, y_train.shape)
print(X_test_province.shape, y_test.shape)
print(vectorizer.get_feature_names())

After vectorizations
(55356, 374) (55356, 1)
(27266, 374) (27266, 1)
['aarde', 'aconcagua', 'aegean', 'africa', 'agioritikos', 'ahr', 'alenquer', 'alentejano', 'alentejo', 'algarve', 'alsace', 'alto', 'america', 'and', 'andalucia', 'antonio', 'apalta', 'arizona', 'atalanti', 'athos', 'atlantico', 'atlantida', 'australia', 'austria', 'awatere', 'baden', 'bairrada', 'bandeira', 'bay', 'beaujolais', 'beira', 'beiras', 'bekaa', 'black', 'bordeaux', 'bot', 'brazil', 'brda', 'british', 'bucelas', 'buin', 'bulgaria', 'burgenland', 'burgundy', 'bío', 'cachapoal', 'cahul', 'california', 'campanha', 'canelones', 'canterbury', 'cape', 'cappadocia', 'carnuntum', 'carolina', 'casablanca', 'catalonia', 'catarina', 'cauquenes', 'cederberg', 'central', 'cephalonia', 'ceres', 'chalkidiki', 'champagne', 'chile', 'choapa', 'claro', 'clemente', 'coast', 'coastal', 'colchagua', 'colorado', 'columbia', 'commandaria', 'constantia', 'corinth', 'costa', 'crete', 'cruz', 'curicó', 'da', 'dalmatia', 'dan', 'danu

# Normalizing the numerical feature price and points

In [21]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['price'].values.reshape(-1,1))
X_train_price = normalizer.transform( X_train['price'].values.reshape(-1,1))
X_test_price=  normalizer.transform(X_test['price'].values.reshape(-1,1))
normalizer.fit(X_train['points'].values.reshape(-1,1))
X_train_points =normalizer.transform( X_train['points'].values.reshape(-1,1))
X_test_points= normalizer.transform(X_test['points'].values.reshape(-1,1))

## Stacking the data for training and testing

In [22]:

from scipy.sparse import hstack
X_tr = hstack((X_train_country,X_train_points,X_train_price,X_train_province,X_train_review,X_train_title,X_train_winery)).tocsr()
X_te=hstack((X_test_country,X_test_points,X_test_price,X_test_province,X_test_review,X_test_title,X_test_winery)).tocsr()
print("Final Data matrix")
print(X_tr.shape, y_train.shape)
print(X_te.shape, y_test.shape)

Final Data matrix
(55356, 22028) (55356, 1)
(27266, 22028) (27266, 1)


# Creating various model for prediction


In [16]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
tuned_parameters = [{'n_neighbors': [3,5,12]}]
model = GridSearchCV(KNeighborsClassifier(), tuned_parameters, scoring = 'accuracy', cv=3)
model.fit(X_tr, y_train)
print(model.best_estimator_)
print("The accuracy of KNN model is: ",model.score(X_te, y_test))


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:531: DataConversionWarning: A column-vector y was passed when

KNeighborsClassifier(n_neighbors=3)
The accuracy of KNN model is:  0.681764835326047


In [ ]:
from sklearn.svm import SVC
C_range = [10**-4, 10**-2, 10**2]
gamma_range = [10**-2,10**2]
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=3)
grid.fit(X_tr,y_train)
print(grid.best_estimator_)
print("The accuracy of SVM model is: ",grid.score(X_te, y_test))


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [15]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
tuned_parameters = [{'C': [10**-4, 10**-2, 10**0, 10**2]}]
model = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy', cv=3)
model.fit(X_tr, y_train)
print(model.best_estimator_)
print("The accuracy of LogisticRegression model is: ",model.score(X_te, y_test))


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

LogisticRegression(C=100)
The accuracy of LogisticRegression model is:  0.9748404606469596


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
from sklearn.ensemble import RandomForestClassifier
tuned_parameters = [{'max_depth':[5,10,12]}]
model = GridSearchCV(RandomForestClassifier(), tuned_parameters, scoring = 'accuracy', cv=3)
model.fit(X_tr, y_train)
print(model.best_estimator_)
print("The accuracy of LogisticRegression model is: ",model.score(X_te, y_test))
 

RandomForestClassifier(max_depth=12)
The accuracy of LogisticRegression model is:  0.749468202156532


In [14]:
from sklearn.ensemble import GradientBoostingClassifier
grid=GradientBoostingClassifier(max_depth=5,n_estimators=70)
grid.fit(X_tr,y_train)
print(grid.score(X_te,y_test))

0.9708061321792709


## Stacking the various model

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
clf1=KNeighborsClassifier(n_neighbors=3)
clf2=LogisticRegression(C=100,max_iter=5000)
clf3=RandomForestClassifier(max_depth=12)
clf4=GradientBoostingClassifier(max_depth=5,n_estimators=70)
lr=LogisticRegression(max_iter=5000)


In [15]:
import mlxtend
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3,clf4], 
                          meta_classifier=clf4)
sclf.fit(X_tr,y_train.ravel())
print(sclf.score(X_te,y_test.ravel()))

0.9731167021198562


In [16]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3,clf4], 
                          meta_classifier=lr)
sclf.fit(X_tr,y_train.ravel())
print(sclf.score(X_te,y_test.ravel()))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.805728746424118


In [17]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3,clf4], 
                          meta_classifier=clf2)
sclf.fit(X_tr,y_train.ravel())
print(sclf.score(X_te,y_test.ravel()))

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9294726032421331


In [30]:
!pip install mlxtend
import mlxtend
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3,clf4], 
                          meta_classifier=clf4)
sclf.fit(X_tr,y_train.ravel())
print(sclf.score(X_te,y_test.ravel()))

     |████████████████████████████████| 1.3 MB 4.7 MB/s eta 0:00:01
0.9723465121396612
